In [2]:
import os
import bs4
import re
import pandas as pd
import numpy as np
import datetime
from collections import Counter

files = os.listdir('Ikon/tmp')

In [3]:
def add_fields(sp, tag, cls):
    
    res = sp.findAll(tag, {'class' : cls})
    if len(res) > 0:
        return res[0].text.strip()
    else:
        return None
    
 
    
all_exhibitions = []   
all_artists = []
all_galleries = []

names_dates = {}

already_processed = set()

for fn in files:
    
    if 2 == 2:
    #if 'IkOn2015.01.26.Mon-2015.02.01.Sun' in fn:
        
        with open('Ikon/tmp/' + fn) as myfile:
            html = myfile.read()
            soup = bs4.BeautifulSoup(html,"lxml")
            exbhis = soup.findAll('div', {'class' : "row"})
            
            for ex in exbhis:
                
                exhibition = {}
        
                year_m  = add_fields(ex, 'div', 'year-month')
                
                if year_m:
                
                    title = add_fields(ex, 'a',   'title')
                
                    if title not in already_processed:

                        already_processed.add(title)

                        day     = add_fields(ex, 'div', 'day')
                        hour    = add_fields(ex, 'div', 'hour')
                        year_m  = ''.join([s for s in year_m if (s == '.' or s.isdigit())])
                        date    = datetime.datetime.strptime(year_m + ' ' + day + ' ' + hour, '%Y.%m. %d %H:%M')
                        gallery = add_fields(ex, 'a',   'gallery')

                        exhibition['date']    = date
                        exhibition['title']   = title
                        exhibition['gallery'] = gallery
                        artists               = ex.findAll('', {'class' : 'artists'}) 
                        exhibition['artists'] =  '; '.join([a.text.replace(';' , ' ') for a in artists])
                        all_exhibitions.append(exhibition)

                        
                        

                        all_artists += [a.text for a in artists]
                        
                        for a in artists:
                            if a.text not in names_dates:
                                names_dates[a.text] = []
                            names_dates[a.text].append(date)
                        
                        all_galleries.append(gallery)
                    

df = pd.DataFrame(all_exhibitions)                    

In [23]:
fout = open('Ikon/extracted/names_min_max_date.dat', 'w')
for n, dates in names_dates.items():
    if len(n) > 0 and len(dates) > 0:
        fout.write(n.encode('utf-8').strip() + '\t' + str(min(dates)) + '\t' + str(max(dates)) + '\n' )
fout.close()

In [25]:
folderout = 'extracted'
if not os.path.exists(folderout):
    os.makedirs(folderout)
    
df = df.drop_duplicates()    
    
len(df)

13265

In [28]:
df.to_csv('Ikon/extracted/ikon_exhibitions_raw.csv', sep = '\t',encoding='utf-8')

fout = open('Ikon/extracted/all_artists_raw.dat', 'w')
all_artists_s = sorted(list(set(all_artists)))
for a in all_artists_s:
    fout.write(a.encode('utf-8').strip()+'\n')
fout.close()


fout = open('Ikon/extracted/all_galleries_raw.dat', 'w')
all_galleries_s = sorted(list(set(all_galleries)))
for a in all_galleries_s:
    fout.write(a.encode('utf-8').strip()+'\n')
fout.close()

In [31]:
artists_cnt = Counter(all_artists)
artists_cnt

Counter({u'': 8,
         u'Sz\u0151ll\u0151ssy Bal\xe1zs,': 3,
         u'Roland Grabkowski,': 2,
         u'Szily G\xe9za,': 2,
         u'V\xe9gi P\xe9ter,': 3,
         u'Szemad\xe1m Gy\xf6rgy Munk\xe1csy-d\xedjas fest\u0151m\u0171v\xe9sz  \xedr\xf3': 1,
         u'M\xfcllner Andr\xe1s irodalm\xe1r,': 2,
         u'SI-LA-GI \xe9s AI WEIWEI,': 1,
         u'Rajcs\xf3k Attila,': 11,
         u'Czidor \xc1gnes,': 6,
         u'Varga Zsuzsa m\u0171v\xe9szett\xf6rt\xe9n\xe9sz': 1,
         u'Benedek Elek,': 2,
         u'Mirte Berko Mallory,': 2,
         u'Karas David': 3,
         u'M\xe1rcio-Andr\xe9': 2,
         u'P\xe1ldi L\xedvia': 13,
         u'P\xe9terfy \xc1bel': 2,
         u'Szekeres G\xe1bor': 2,
         u'He\xadge\xadd\xfcs \xc9va,': 2,
         u'V\xe9s\u0151 \xc1goston,': 1,
         u'Iszkaszentgy\xf6rgy,': 2,
         u'Franz HANFSTAENGL,': 2,
         u'Antim\xe9dia': 2,
         u'RO m.v.,': 2,
         u'Olgyai Viktor,': 2,
         u'Hegyeshalmi L\xe1szl\xf3 a ve

In [32]:
fout = open('Ikon/extracted/all_artists_cnt.dat', 'w')
artists_cnt = Counter(all_artists)
for (a, c) in dict(artists_cnt).items():
    fout.write(a.encode('utf-8').strip() + '\t' + str(c) +'\n')
fout.close()

fout = open('Ikon/extracted/all_galleries_cnt.dat', 'w')
galleries_cnt = Counter(all_galleries)
for a, c in dict(galleries_cnt).items():
    fout.write(a.encode('utf-8').strip() + '\t' + str(c) +'\n')
fout.close()